# Data loading, storage, and file formats
# 数据加载、存储与文件格式

In [1]:
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [2]:
%pwd

'C:\\Users\\MappingLab-lxy\\pydata-book\\branches\\1st-edition'

## Reading and Writing Data in Text Format
## 读写文本格式的数据

In [6]:
#!cat ch06/ex1.csv   
#linux下的命令

In [9]:
df = pd.read_csv('ch06/ex1.csv')#针对逗号分隔符文件的读取
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
pd.read_table('ch06/ex1.csv', sep=',')#读取带有分隔符的文件，默认分隔符为'\t'

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
#!cat ch06/ex2.csv


In [22]:
print(pd.read_csv('ch06/ex2.csv', header=None))#没有表头，否则会把第一行当做表头读取
pd.read_csv('ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

   0   1   2   3      4
0  1   2   3   4  hello
1  5   6   7   8  world
2  9  10  11  12    foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [24]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ch06/ex2.csv', names=names, index_col='message')#指定列作为行索引

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [35]:
!type ch06\csv_mindex.csv
parsed = pd.read_csv('ch06/csv_mindex.csv', index_col=['key1', 'key2'])#指定两列作为层次化索引
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [27]:
list(open('ch06/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [29]:
result = pd.read_table('ch06/ex3.txt', sep='\s+')#可以使用正则表达式解析分隔符，由于列名的数量比列的数量少1，因此推断第一列为行索引
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [38]:
!type ch06\ex4.csv
pd.read_csv('ch06/ex4.csv', skiprows=[0, 2, 3])#使用skiprows跳过某些行

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [43]:
!type ch06\ex5.csv
result = pd.read_csv('ch06/ex5.csv')
result

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [42]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [54]:
result = pd.read_csv('ch06/ex5.csv', na_values=['foo'])
#na_values 用于替换NA/NaN的值。原本为foo的值现在被设为了NaN
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [55]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('ch06/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading text files in pieces
### 逐块读取文本文件

In [56]:
result = pd.read_csv('ch06/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [58]:
pd.read_csv('ch06/ex6.csv', nrows=5)#读取前n行

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [60]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)#迭代读取，每次读取的行数为1000
chunker

In [64]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)#使用series的key对齐相加，原本若干行的代码一行即可

tot = tot.sort_values(ascending=False)

In [65]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing data out to text format
### 将数据写出到文本格式

In [66]:
data = pd.read_csv('ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [67]:
data.to_csv('ch06/out.csv')
!type ch06\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [69]:
data.to_csv(sys.stdout, sep='|')#sys.stdout直接打印出来，并没有真的写出到文件

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [71]:
data.to_csv(sys.stdout, na_rep='NULL')#将缺失值表示为NULL

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [73]:
data.to_csv(sys.stdout, index=False, header=False)#不输出行/列索引

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [74]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [77]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('ch06/tseries.csv')#series也有导出至文本文件的方法
!type ch06\tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [78]:
Series.from_csv('ch06/tseries.csv', parse_dates=True)#series对应的方法直接读取，不用read_csv

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### Manually working with delimited formats
### 手动处理分隔符格式

In [85]:
!type ch06\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [88]:
import csv
f = open('ch06/ex7.csv')

reader = csv.reader(f)#使用python内置的csv模块读取

In [89]:
for line in reader:
    print(line)
f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [90]:
lines = list(csv.reader(open('ch06/ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}#手动组装一个字典
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [92]:
#定义csv.Dialect的一个子类，定义一种解析格式
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [94]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)#根据自定的格式写文件
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [97]:
!type mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


### JSON data
### JSON数据

In [98]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [102]:
import json
result = json.loads(obj)#null被load为了None
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'Cisco'}]}

In [103]:
asjson = json.dumps(result)#dumps将python数据转为json型数据

In [106]:
siblings = DataFrame(result['siblings'])#将json型数据的子集传给DataFrame构造器
siblings

,age,name,pet
0,25,Scott,Zuko
1,33,Katie,Cisco


### XML and HTML, Web scraping
### XML 和HTML:网络数据的收集

>**NB. The Yahoo! Finance API has changed and this example no longer works**

**自己根据网页结构解析一个网页**


In [136]:
from lxml.html import parse
from urllib.request import urlopen

parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))#获取苹果数据

doc = parsed.getroot()#获取根元素
print(doc)

<Element html at 0x232482bec78>


In [137]:
links = doc.findall('.//a')#XPath的用法，选取当前节点中所有 a的元素
lnk_text = [lnk.text_content() for lnk in links]

In [138]:
lnk = links[28]
lnk
lnk.get('href')
lnk.text_content()

'142.00'

In [139]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['/',
 '/watchlists',
 '/portfolios?bypass=true',
 '/screener',
 '/calendar',
 '/industries',
 '/personal-finance',
 '/tech',
 '/topic/yahoo-finance-podcast',
 '/live/facebook']

In [140]:
tables = doc.findall('.//table')
tables
calls = tables[0]
puts = tables[1]

In [141]:
rows = calls.findall('.//tr')

In [142]:
def _unpack(row, kind='td'):#默认获取数据行，根据传参不同获得不同信息
    elts = row.findall('.//%s' % kind)#XPath获取tr or td
    return [val.text_content() for val in elts]#获取单元格的文本

In [143]:
_unpack(rows[0], kind='th')
_unpack(rows[1], kind='td')

['AAPL171222C00115000',
 '2017-12-21 3:38PM EST',
 '115.00',
 '60.40',
 '59.90',
 '60.15',
 '+5.80',
 '+10.62%',
 '20',
 '710',
 '284.38%']

In [144]:
from pandas.io.parsers import TextParser

def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')#获取表头
    data = [_unpack(r) for r in rows[1:]]#获取数据
    return TextParser(data, names=header).get_chunk()#TextParser自动解析数据类型

In [145]:
call_data = parse_options_data(calls)
put_data = parse_options_data(puts)
call_data[:10]

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,AAPL171222C00115000,2017-12-21 3:38PM EST,115.0,60.40,59.90,60.15,5.80,+10.62%,20,710,284.38%
1,AAPL171222C00120000,2017-12-21 3:38PM EST,120.0,55.40,54.90,55.15,-1.55,-2.72%,13,107,256.25%
2,AAPL171222C00125000,2017-12-19 11:42AM EST,125.0,49.79,49.90,50.15,0.00,-,20,50,231.25%
3,AAPL171222C00130000,2017-12-21 9:50AM EST,130.0,45.11,44.55,45.30,1.02,+2.31%,123,966,304.69%
4,AAPL171222C00135000,2017-12-20 11:18AM EST,135.0,38.81,39.55,40.45,0.00,-,12,29,293.36%
5,AAPL171222C00140000,2017-12-11 10:32AM EST,140.0,30.87,32.95,35.00,0.00,-,30,0,0.00%
6,AAPL171222C00142000,2017-12-20 9:58AM EST,142.0,31.80,32.55,33.30,0.00,-,65,65,226.17%
7,AAPL171222C00143000,2017-12-20 9:58AM EST,143.0,30.84,31.65,32.30,0.00,-,70,70,219.92%
8,AAPL171222C00144000,2017-12-20 11:22AM EST,144.0,29.71,30.55,31.35,0.00,-,75,25,220.12%
9,AAPL171222C00145000,2017-12-20 11:22AM EST,145.0,28.79,29.80,30.30,0.00,-,80,40,153.13%


#### Parsing XML with lxml.objectify
#### 使用lxml.objectify解析XML

In [148]:
%cd ./ch06/mta_perf/

C:\Users\MappingLab-lxy\pydata-book\branches\1st-edition\ch06\mta_perf


In [150]:
!type -21 Performance_MNR.xml

<?xml  version="1.0" encoding="ISO-8859-1"?>
<PERFORMANCE>
<INDICATOR>
  <INDICATOR_SEQ>28445</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Performance (West of Hudson)</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.
</DESCRIPTION>
  <PERIOD_YEAR>2008</PERIOD_YEAR>
  <PERIOD_MONTH>1</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>95.00</YTD_TARGET>
  <YTD_ACTUAL>96.90</YTD_ACTUAL>
  <MONTHLY_TARGET>95.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>96.90</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICAT

  <INDICATOR_NAME>Pascack Valley Line - OTP</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. Metro-North Railroad contracts with New Jersey Transit to operate service on the Pascack Valley Line.</DESCRIPTION>
  <PERIOD_YEAR>2008</PERIOD_YEAR>
  <PERIOD_MONTH>7</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>95.60</YTD_TARGET>
  <YTD_ACTUAL>96.80</YTD_ACTUAL>
  <MONTHLY_TARGET>95.60</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>97.70</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28461</INDICATOR_SEQ>
  <PARENT_SEQ>28445</PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Pascack Valley Line - OTP</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minu

  <DECIMAL_PLACES>2</DECIMAL_PLACES>
  <YTD_TARGET>2.00</YTD_TARGET>
  <YTD_ACTUAL>1.24</YTD_ACTUAL>
  <MONTHLY_TARGET>2.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>1.03</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28627</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Employee Lost Time and Restricted Duty Rate</INDICATOR_NAME>
  <DESCRIPTION>An employee lost time injury or illness is one that prevents an employee from returning to work for at least one full shift. The rate is injuries and illnesses per 200,000 worker hours.</DESCRIPTION>
  <PERIOD_YEAR>2008</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Safety Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>D</DESIRED_CHANGE>
  <INDICATOR_UNIT>-</INDICATOR_UNIT>
  <DECIMAL_PLACES>2</DECIMAL_PLACES>
  <YTD_TARGET>2.00</YTD_TARGET>
  <YTD_ACTUAL>1.22</YTD_ACTUAL>
  <MONTHLY_TARGET>2.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>.79</MONTHLY_A

  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Hudson Line - OTP</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time.</DESCRIPTION>
  <PERIOD_YEAR>2009</PERIOD_YEAR>
  <PERIOD_MONTH>8</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.90</YTD_TARGET>
  <YTD_ACTUAL>97.90</YTD_ACTUAL>
  <MONTHLY_TARGET>97.90</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>96.70</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28345</INDICATOR_SEQ>
  <PARENT_SEQ>55526</PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Hudson Line - OTP</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time.</DESCRIPTION>
  <PERIOD_YE

  <INDICATOR_SEQ>55526</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Performance (East of Hudson)</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. East of Hudson service includes the Harlem, Hudson and New Haven lines.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>5</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.80</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.80</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>55526</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Performance (East of Hudson)</INDICATOR_N

系统找不到指定的文件。
处理: -21 时出错。

Performance_MNR.xml




In [156]:
from lxml import objectify

path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path))#通过objectify.parse解析文件
root = parsed.getroot()

In [157]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']
print(root.INDICATOR[:10])
for elt in root.INDICATOR:#XPath的节点
    el_data = {}
    for child in elt.getchildren():#在节点的子节点中
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval#标签=值
    data.append(el_data)

[<Element INDICATOR at 0x23246f00e08>, <Element INDICATOR at 0x2324830e688>, <Element INDICATOR at 0x232482b1b88>, <Element INDICATOR at 0x232483f4548>, <Element INDICATOR at 0x232483bd308>, <Element INDICATOR at 0x232483bd388>, <Element INDICATOR at 0x232483bd408>, <Element INDICATOR at 0x232483bd448>, <Element INDICATOR at 0x232483bd4c8>, <Element INDICATOR at 0x232483bd108>]


In [159]:
perf = DataFrame(data)#通过XML整理出了一个表格
perf.iloc[:10]

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


In [168]:
import io
tag = '<a href="http://www.google.com">Google</a>'
root= objectify.parse(io.StringIO(tag)).getroot()

In [169]:
root

<Element a at 0x2324859e688>

In [170]:
root.get('href')

'http://www.google.com'

In [171]:
root.text

'Google'

## Binary data formats
## 二进制数据格式

In [172]:
cd ../..

C:\Users\MappingLab-lxy\pydata-book\branches\1st-edition


In [176]:
frame = pd.read_csv('ch06/ex1.csv')
print(frame)
frame.to_pickle('ch06/frame_pickle')#pickle可以很方便的把数据以二进制形式保存

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [177]:
pd.read_pickle('ch06/frame_pickle')#读取数据

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### Using HDF5 format
### 使用HDF5格式
HDF=层次型数据格式   
+ HDF5文件含有一个文件系统式的节点结构，能够存储多个数据集且支持元数据
+ 支持多种压缩器即时压缩
+ 高效存储重复数据集
**适合做数据集，适合一次写多次读**

In [183]:
store = pd.HDFStore('mydata.h5')#类似字典的结构
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
/obj1                frame        (shape->[3,5])
/obj1_col            series       (shape->[3])  

In [184]:
store['obj1']

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [185]:
store.close()
os.remove('mydata.h5')

PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'mydata.h5'

### Interacting with HTML and Web APIs
### 使用HTML和Web API

In [189]:
import requests
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
resp = requests.get(url)#直接发送一个get请求
resp

<Response [200]>

In [193]:
import json
data = DataFrame(json.loads(resp.text))
data

,color,default,id,name,url
0,e10c02,False,76811,Bug,https://api.github.com/repos/pandas-dev/pandas...
1,4E9A06,False,76812,Enhancement,https://api.github.com/repos/pandas-dev/pandas...
2,FCE94F,False,127681,Refactor,https://api.github.com/repos/pandas-dev/pandas...
3,75507B,False,129350,Build,https://api.github.com/repos/pandas-dev/pandas...
4,3465A4,False,134699,Docs,https://api.github.com/repos/pandas-dev/pandas...
5,AFEEEE,False,211840,Timeseries,https://api.github.com/repos/pandas-dev/pandas...
6,729FCF,False,233160,Groupby,https://api.github.com/repos/pandas-dev/pandas...
7,06909A,False,2301354,Data IO,https://api.github.com/repos/pandas-dev/pandas...
8,8AE234,False,2413328,Visualization,https://api.github.com/repos/pandas-dev/pandas...
9,0b02e1,False,2822098,Indexing,https://api.github.com/repos/pandas-dev/pandas...


## Interacting with databases
## 使用数据库

In [195]:
import sqlite3
#创建一个名为test的表，列a 最长20char...
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [197]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)#插入数据库
con.commit()

In [198]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [217]:
DataFrame(rows, columns=(next(zip(*cursor.description))))

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5


In [218]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
